import dependencies

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\damindu pahasara\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


data pre-processing

In [2]:
#loading csv's data to pandas data frame
movies_data = pd.read_csv("movies.csv")

In [3]:
tvs_data = pd.read_csv("TMDB_tv_dataset_v3.csv")

In [4]:
tvs_data.head()

,id,name,number_of_seasons,number_of_episodes,original_language,vote_count,vote_average,overview,adult,backdrop_path,...,tagline,genres,created_by,languages,networks,origin_country,spoken_languages,production_companies,production_countries,episode_run_time
0,1399,Game of Thrones,8,73,en,21857,8.442,Seven noble families fight for control of the ...,False,/2OMB0ynKlyIenMJWI2Dy9IWT4c.jpg,...,Winter Is Coming,"Sci-Fi & Fantasy, Drama, Action & Adventure","David Benioff, D.B. Weiss",en,HBO,US,English,"Revolution Sun Studios, Television 360, Genera...","United Kingdom, United States of America",0
1,71446,Money Heist,3,41,es,17836,8.257,"To carry out the biggest heist in history, a m...",False,/gFZriCkpJYsApPZEF3jhxL4yLzG.jpg,...,The perfect robbery.,"Crime, Drama",Álex Pina,es,"Netflix, Antena 3",ES,Español,Vancouver Media,Spain,70
2,66732,Stranger Things,4,34,en,16161,8.624,"When a young boy vanishes, a small town uncove...",False,/2MaumbgBlW1NoPo3ZJO38A6v7OS.jpg,...,Every ending has a beginning.,"Drama, Sci-Fi & Fantasy, Mystery","Matt Duffer, Ross Duffer",en,Netflix,US,English,"21 Laps Entertainment, Monkey Massacre Product...",United States of America,0
3,1402,The Walking Dead,11,177,en,15432,8.121,Sheriff's deputy Rick Grimes awakens from a co...,False,/x4salpjB11umlUOltfNvSSrjSXm.jpg,...,Fight the dead. Fear the living.,"Action & Adventure, Drama, Sci-Fi & Fantasy",Frank Darabont,en,AMC,US,English,"AMC Studios, Circle of Confusion, Valhalla Mot...",United States of America,42
4,63174,Lucifer,6,93,en,13870,8.486,"Bored and unhappy as the Lord of Hell, Lucifer...",False,/aDBRtunw49UF4XmqfyNuD9nlYIu.jpg,...,It's good to be bad.,"Crime, Sci-Fi & Fantasy",Tom Kapinos,en,"FOX, Netflix",US,English,"Warner Bros. Television, DC Entertainment, Jer...",United States of America,45


In [3]:
#print the first fve rows from the data frame
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
#rows and colomn count
movies_data.shape

(4803, 24)

In [5]:
#selescting freatures for recomdations (selescting coloumns)
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [6]:
#replace the null values with null strings
for freature in selected_features:
  movies_data[freature] = movies_data[freature].fillna('')

In [7]:
#combine all seleted features
combined_features = movies_data['genres']+''+movies_data['keywords']+''+movies_data['tagline']+''+movies_data['cast']+''+movies_data['director']
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [8]:
#convert text data to feature vectors
vectorizer = TfidfVectorizer()

In [9]:
feature_vectros = vectorizer.fit_transform(combined_features)

In [10]:
print(feature_vectros)

  (0, 3583)	0.16941894714909375
  (0, 20729)	0.2738578969459924
  (0, 16614)	0.15696073877453268
  (0, 14550)	0.2236809363648068
  (0, 23267)	0.16128139780622516
  (0, 26547)	0.19638671147741732
  (0, 22464)	0.20207089568711745
  (0, 21487)	0.21550128478931546
  (0, 27540)	0.19771357974524176
  (0, 27182)	0.23480088356130552
  (0, 21521)	0.1562356665943368
  (0, 18558)	0.2612168335104853
  (0, 18164)	0.08690831799482265
  (0, 27161)	0.1261988657937151
  (0, 24197)	0.07518543993419265
  (0, 22778)	0.2738578969459924
  (0, 4676)	0.2452909770424961
  (0, 26334)	0.13059723071916296
  (0, 22916)	0.33668756406929184
  (0, 10193)	0.16532432420812987
  (0, 4456)	0.21799675215510664
  (0, 9304)	0.2738578969459924
  (0, 21836)	0.09966592997173944
  (0, 8936)	0.11806131645084653
  (0, 444)	0.09109355212252294
  :	:
  (4801, 5449)	0.31510541707008866
  (4801, 22170)	0.31510541707008866
  (4801, 27361)	0.31510541707008866
  (4801, 27513)	0.3005604008026175
  (4801, 8121)	0.2611505149930138
  (4801,

In [11]:
#getting the simillarity scores using cosine simirarity
similarity = cosine_similarity(feature_vectros)
print (similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [12]:
#geting user input (movie name)
movie_name = input('Enter movie name: ')

Enter movie name: cars


In [13]:
#create all movie lidt
LIst_of_all_titles = movies_data['title'].tolist()
print(LIst_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [14]:
#find close name for users given name
find_close_match = difflib.get_close_matches(movie_name, LIst_of_all_titles)
print (find_close_match)

['Cars', 'Takers', 'Oceans']


In [15]:
#find most close name for user given name
close_match = find_close_match[0]
print(close_match)

Cars


In [16]:
#find the movie index with title
index_of_the_movie = movies_data[movies_data.title==close_match]['index'].values[0]
print(index_of_the_movie)

566


In [17]:
#geting list of similer movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.012873680847883073), (1, 0.02707089418013921), (2, 0.00789455888518013), (3, 0.0), (4, 0.007504289922933021), (5, 0.0), (6, 0.0411144525080715), (7, 0.02499572866350348), (8, 0.00807725361290275), (9, 0.007728882656213176), (10, 0.014873647681014141), (11, 0.007106526739758839), (12, 0.007811548494856592), (13, 0.012473200608735153), (14, 0.03316081979760274), (15, 0.024066030530467044), (16, 0.0), (17, 0.007535277229544703), (18, 0.009906760806917141), (19, 0.015868062611147912), (20, 0.007408543367434388), (21, 0.018646423851578258), (22, 0.012320535352035738), (23, 0.015609556553785483), (24, 0.01209069737249866), (25, 0.0), (26, 0.008194301579369666), (27, 0.006926723868974821), (28, 0.007455808574665135), (29, 0.008113409657334812), (30, 0.007662749919436539), (31, 0.04786727920947036), (32, 0.0), (33, 0.007774911696866594), (34, 0.030632336163044664), (35, 0.02326125647485454), (36, 0.012854021493776707), (37, 0.022013036155600154), (38, 0.00783724630196297), (39, 0.007615

In [18]:
#sorting movies to Z-A
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(566, 1.0000000000000004), (40, 0.3116582947611961), (405, 0.16932965758291965), (3318, 0.14231052396012717), (2426, 0.13046583212391613), (99, 0.12273310242688143), (500, 0.11643566720789728), (1157, 0.11494364321351744), (1186, 0.11425811719240889), (3643, 0.11266113705164797), (1022, 0.11140624038239261), (3289, 0.10760161596587195), (409, 0.10649882137526384), (3526, 0.10469816421288299), (1283, 0.1015998335354126), (706, 0.0978259244361549), (658, 0.09719716604739381), (177, 0.09699766997992086), (226, 0.09557206543046448), (3447, 0.0947103730562075), (4705, 0.09282597129704748), (384, 0.09218670105267363), (1152, 0.09124217830645742), (4235, 0.0902123350277142), (4239, 0.08972869763366258), (1541, 0.08806228685676948), (3030, 0.08619977744065667), (273, 0.08463353389853578), (707, 0.08282425424995501), (2535, 0.08225950333617865), (3287, 0.08161329868939342), (3606, 0.08133192503740541), (1999, 0.0810405041927135), (3315, 0.07941151480632391), (2247, 0.07831950464780506), (690, 

In [19]:
#print similar movies based on user input( using index)
i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<21):
    print(i, '.',title_from_index)
    i+=1

1 . Cars
2 . Cars 2
3 . The Fast and the Furious: Tokyo Drift
4 . Witless Protection
5 . Larry the Cable Guy: Health Inspector
6 . The Fast and the Furious
7 . 2 Fast 2 Furious
8 . Cheaper by the Dozen
9 . The Final Destination
10 . Bottle Rocket
11 . The Cable Guy
12 . The Wendell Baker Story
13 . All That Jazz
14 . The Sting
15 . The Hudsucker Proxy
16 . Days of Thunder
17 . Death Race
18 . Turbo
19 . How Do You Know
20 . Butch Cassidy and the Sundance Kid


# **movie recommondation system**

In [20]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : cars
Movies suggested for you : 

1 . Cars
2 . Cars 2
3 . The Fast and the Furious: Tokyo Drift
4 . Witless Protection
5 . Larry the Cable Guy: Health Inspector
6 . The Fast and the Furious
7 . 2 Fast 2 Furious
8 . Cheaper by the Dozen
9 . The Final Destination
10 . Bottle Rocket
11 . The Cable Guy
12 . The Wendell Baker Story
13 . All That Jazz
14 . The Sting
15 . The Hudsucker Proxy
16 . Days of Thunder
17 . Death Race
18 . Turbo
19 . How Do You Know
20 . Butch Cassidy and the Sundance Kid
21 . The Woman Chaser
22 . Cast Away
23 . Back to the Future Part II
24 . I Want Someone to Eat Cheese With
25 . Stolen Summer
26 . Toy Story
27 . Tremors
28 . Gone in Sixty Seconds
29 . Cheaper by the Dozen 2
